<a href="https://colab.research.google.com/github/wanadzhar913/bank-transaction-classification-debertav3/blob/master/notebooks/02_model_building_deberta_with_non_text_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install datasets huggingface_hub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# import transformers
# import datasets
# import huggingface_hub
# import sklearn
# import torch
# import numpy as np
# import pandas as pd

# print(f'transformers version: {transformers.__version__}')
# print(f'datasets version: {datasets.__version__}')
# print(f'huggingface_hub version: {huggingface_hub.__version__}')
# print(f'scikit-learn version: {sklearn.__version__}')
# print(f'torch version: {torch.__version__}')
# print(f'numpy version: {np.__version__}')
# print(f'pandas version: {pd.__version__}')

transformers version: 4.47.1
datasets version: 3.2.0
huggingface_hub version: 0.27.1
scikit-learn version: 1.6.0
torch version: 2.5.1+cu121
numpy version: 1.26.4
pandas version: 2.2.2


In [ ]:
import os
import json
import random
from typing import Optional, Union, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from datasets import load_dataset, Dataset
from huggingface_hub import create_repo, notebook_login
from sklearn.metrics import classification_report, average_precision_score, \
                            accuracy_score, f1_score, precision_score, recall_score

import torch
from torch import nn
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import AutoTokenizer, AutoConfig, DebertaV2Model, \
                         DebertaV2ForSequenceClassification

%matplotlib inline

In [ ]:
notebook_login()

##### 1.0 Load dataset

In [ ]:
!mkdir data/
!cd data/ && wget https://raw.githubusercontent.com/wanadzhar913/bank-transaction-classification/refs/heads/master/data/train.csv -q
!cd data/ && wget https://raw.githubusercontent.com/wanadzhar913/bank-transaction-classification/refs/heads/master/data/test.csv -q

In [ ]:
train = pd.read_csv('data/train.csv')
train.head(3)

,client_id,bank_id,account_id,txn_id,txn_date,description,amount,description_stem,description_clean_len,day_monday,...,monthly_transaction_count_atm,monthly_transaction_count_bank_fee,monthly_transaction_count_payment,IS_INTERESTED_INVESTMENT,IS_INTERESTED_BUILD_CREDIT,IS_INTERESTED_INCREASE_INCOME,IS_INTERESTED_PAY_OFF_DEBT,IS_INTERESTED_MANAGE_SPENDING,IS_INTERESTED_GROW_SAVINGS,category
0,577,1,2,90,2023-08-14 00:00:00,Online Banking transfer from CHK 6479 Confirma...,6.0,onlin bank transfer chk confirm,5,1,...,0,0,0,0,0,1,0,1,0,Internal Account Transfer
1,94,8,8,159,2023-09-18 00:00:00,DEPOSIT,5.0,deposit,1,1,...,0,0,0,0,0,0,0,0,0,Payroll
2,880,770,887,99720,2023-06-16 00:00:00,Bulk 7202 Apple 1920 EAST WISCONSI APPLETON ...,-10.9,bulk appl east wisconsi appleton wiu,6,0,...,1891,12,14,0,0,0,0,0,0,Uncategorized


In [ ]:
test = pd.read_csv('data/test.csv')
test.head(3)

,client_id,bank_id,account_id,txn_id,txn_date,description,amount,description_stem,description_clean_len,day_monday,...,monthly_transaction_count_atm,monthly_transaction_count_bank_fee,monthly_transaction_count_payment,IS_INTERESTED_INVESTMENT,IS_INTERESTED_BUILD_CREDIT,IS_INTERESTED_INCREASE_INCOME,IS_INTERESTED_PAY_OFF_DEBT,IS_INTERESTED_MANAGE_SPENDING,IS_INTERESTED_GROW_SAVINGS,category
0,880,828,953,24000,2023-08-11 00:00:00,SECU Foundation,-0.200,secu foundat,2,0,...,1891,12,14,0,0,0,0,0,0,Restaurants
1,880,17,17,178308,2023-09-26 00:00:00,SELF LENDER AUSTIN TX 23267 Debit Card Purchas...,-5.240,self lender austin tx debit card purchas,7,0,...,1891,12,14,0,0,0,0,0,0,Uncategorized
2,880,489,558,89624,2023-06-05 00:00:00,RECURRING PAYMENT AUTHORIZED ON 06/02 GEICO *A...,-50.398,recur payment author geico auto dc,6,1,...,1891,12,14,0,0,0,0,0,0,Insurance


In [ ]:
# to ensure the presence of every class in each dataset split
assert train.category.nunique() == test.category.nunique(), 'Num. of categories are different'

category = list(train.category.unique())

# Sorting the categories alphabetically
sorted_categories = sorted(category)

# Generating the category mapping dictionary with values starting from 0
category_dict = {category: index for index, category in enumerate(sorted_categories)}
category_dict

{'ATM': 0,
 'Arts and Entertainment': 1,
 'Bank Fee': 2,
 'Bank Fees': 3,
 'Check Deposit': 4,
 'Clothing and Accessories': 5,
 'Convenience Stores': 6,
 'Department Stores': 7,
 'Digital Entertainment': 8,
 'Food and Beverage Services': 9,
 'Gas Stations': 10,
 'Gyms and Fitness Centers': 11,
 'Healthcare': 12,
 'Insurance': 13,
 'Interest': 14,
 'Internal Account Transfer': 15,
 'Loans': 16,
 'Payment': 17,
 'Payroll': 18,
 'Restaurants': 19,
 'Service': 20,
 'Shops': 21,
 'Supermarkets and Groceries': 22,
 'Tax Refund': 23,
 'Telecommunication Services': 24,
 'Third Party': 25,
 'Transfer': 26,
 'Transfer Credit': 27,
 'Transfer Debit': 28,
 'Transfer Deposit': 29,
 'Travel': 30,
 'Uncategorized': 31,
 'Utilities': 32}

In [ ]:
df_train = Dataset.from_pandas(train)
df_train

Dataset({
    features: ['client_id', 'bank_id', 'account_id', 'txn_id', 'txn_date', 'description', 'amount', 'description_stem', 'description_clean_len', 'day_monday', 'day_friday', 'day_weekend', 'monthly_transaction_count', 'monthly_transaction_count_loans', 'monthly_transaction_count_transfer_credit', 'monthly_transaction_count_transfer_deposit', 'monthly_transaction_count_payroll', 'monthly_transaction_count_uncategorized', 'monthly_transaction_count_restaurants', 'monthly_transaction_count_check_deposit', 'monthly_transaction_count_third_party', 'monthly_transaction_count_food_and_beverage_services', 'monthly_transaction_count_internal_account_transfer', 'monthly_transaction_count_shops', 'monthly_transaction_count_supermarkets_and_groceries', 'monthly_transaction_count_telecommunication_services', 'monthly_transaction_count_bank_fees', 'monthly_transaction_count_utilities', 'monthly_transaction_count_insurance', 'monthly_transaction_count_digital_entertainment', 'monthly_transac

In [ ]:
df_test = Dataset.from_pandas(test)
df_test

Dataset({
    features: ['client_id', 'bank_id', 'account_id', 'txn_id', 'txn_date', 'description', 'amount', 'description_stem', 'description_clean_len', 'day_monday', 'day_friday', 'day_weekend', 'monthly_transaction_count', 'monthly_transaction_count_loans', 'monthly_transaction_count_transfer_credit', 'monthly_transaction_count_transfer_deposit', 'monthly_transaction_count_payroll', 'monthly_transaction_count_uncategorized', 'monthly_transaction_count_restaurants', 'monthly_transaction_count_check_deposit', 'monthly_transaction_count_third_party', 'monthly_transaction_count_food_and_beverage_services', 'monthly_transaction_count_internal_account_transfer', 'monthly_transaction_count_shops', 'monthly_transaction_count_supermarkets_and_groceries', 'monthly_transaction_count_telecommunication_services', 'monthly_transaction_count_bank_fees', 'monthly_transaction_count_utilities', 'monthly_transaction_count_insurance', 'monthly_transaction_count_digital_entertainment', 'monthly_transac

For this experiment, **we'll only take text as our input**. Hence, we're only interested the *description* column for our input features, X.

In [ ]:
 def map_category(example):
    """This is a function to map our categories to unique IDs"""
    example['category'] = category_dict.get(example['category'], -1)  # -1 for unknown categories
    return example

In [ ]:
cols_to_remove = ['client_id', 'bank_id', 'account_id', 'txn_id', 'txn_date',  'description_stem', 'description_clean_len', 'category']

In [ ]:
cols_to_preprocess = sorted(list(set(df_test.column_names) - set(cols_to_remove)))

In [ ]:
def transform_row(row):
    """This is a function to extract the text data and convert our non-text features into a list."""

    # Separate description
    description = row["description"]

    # Collect remaining features and convert to a FloatTensor
    extra_data = [row[key] for key in cols_to_preprocess if key != "description"]

    return {
        "description": description,
        "extra_data": extra_data
    }

In [ ]:
train_X = df_train.remove_columns(cols_to_remove) \
                  .map(transform_row, num_proc=2) \
                  .select_columns(['description', 'extra_data'])
test_X = df_test.remove_columns(cols_to_remove) \
                .map(transform_row, num_proc=2) \
                .select_columns(['description', 'extra_data'])

train_Y = df_train.select_columns(["category"]).map(map_category)
test_Y = df_test.select_columns(["category"]).map(map_category)

Map (num_proc=2):   0%|          | 0/199260 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/49816 [00:00<?, ? examples/s]

Map:   0%|          | 0/199260 [00:00<?, ? examples/s]

Map:   0%|          | 0/49816 [00:00<?, ? examples/s]

In [ ]:
print(train_X[20])
print(train_Y[20])

{'description': 'Transfer From -Graham Dorsa 0323', 'extra_data': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 1.0, 0.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 3.0, 0.0, 2.0, 0.0, 0.0, 0.0]}
{'category': 27}


##### 2.0 Load & set model configs

In [ ]:
class ClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config, num_extra_dims):
        super().__init__()
        total_dims = config.hidden_size+num_extra_dims
        self.dense = nn.Linear(total_dims, total_dims)
        classifier_dropout = config.hidden_dropout_prob
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(total_dims, config.num_labels)

    def forward(self, features, **kwargs):
        x = self.dropout(features)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

In [ ]:
class CustomSequenceClassification(DebertaV2ForSequenceClassification):

    def __init__(self, config, num_extra_dims):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        # might need to rename this depending on the model
        self.deberta =  DebertaV2Model(config)
        self.classifier = ClassificationHead(config, num_extra_dims)

        # Initialize weights and apply final processing
        self.post_init()


    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        extra_data: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.deberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # sequence_output will be (batch_size, seq_length, hidden_size)
        sequence_output = outputs[0]

        # additional data should be (batch_size, num_extra_dims)
        cls_embedding = sequence_output[:, 0, :]

        output = torch.cat((cls_embedding, extra_data), dim=-1)

        logits = self.classifier(output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = nn.MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = nn.BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
model = CustomSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-base",
    num_labels=len(set(category)),
    num_extra_dims=44,
    hidden_dropout_prob=0.1
)

model.cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of CustomSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CustomSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
!nvidia-smi

Sun Jan 12 06:32:50 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              49W / 400W |   1181MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

##### 3.0 Train/finetune model

In [ ]:
trainable_parameters = [param for param in model.parameters() if param.requires_grad]
trainer = torch.optim.AdamW(trainable_parameters, lr = 1e-5, eps=1e-08, betas=(0.9,0.999))

# Add a ReduceLROnPlateau scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(trainer, mode='max', patience=2, factor=0.5, verbose=True)

In [ ]:
batch_size = 64
epoch = 15 # due to compute (and time) constraints, we set a maximum of 15 epochs

best_dev_acc = -np.inf
patient = 3
current_patient = 0

for e in range(epoch):
    pbar = tqdm(range(0, len(train_X), batch_size))
    training_losses = []
    grad_norms = []
    for i in pbar:
        trainer.zero_grad()
        x = train_X[i: i + batch_size]
        y = np.array(train_Y['category'][i: i + batch_size])

        padded = tokenizer(x['description'], padding = 'longest', return_tensors = 'pt')

        padded['labels'] = torch.from_numpy(y).long()
        for k in padded.keys():
            padded[k] = padded[k].cuda()

        padded.pop('token_type_ids', None)

        loss, pred = model(
            **padded,
            extra_data=torch.vstack([torch.FloatTensor([num]) for num in x['extra_data']]).cuda(),
            return_dict = False
        )
        loss.backward()

        grad_norm = torch.nn.utils.clip_grad_norm_(trainable_parameters, 5.0)
        trainer.step()
        training_losses.append(float(loss))
        grad_norms.append(float(grad_norm))

    model.eval()
    dev_predicted = []
    validation_losses = []
    for i in range(0, len(test_X), batch_size):
        x = test_X[i: i + batch_size]
        y = np.array(test_Y['category'][i: i + batch_size])

        padded = tokenizer(x['description'], padding = 'longest', return_tensors = 'pt')
        padded['labels'] = torch.from_numpy(y).long()
        for k in padded.keys():
            padded[k] = padded[k].cuda()

        with torch.no_grad():
            loss, pred = model(
                **padded,
                extra_data=torch.vstack([torch.FloatTensor([num]) for num in x['extra_data']]).cuda(),
                return_dict = False
            )
        dev_predicted.extend(pred.argmax(axis = 1).detach().cpu().numpy().tolist())
        validation_losses.append(float(loss))

    # some notes on the warnings below: https://stackoverflow.com/questions/43162506/undefinedmetricwarning-f-score-is-ill-defined-and-being-set-to-0-0-in-labels-wi
    accuracy = accuracy_score(dev_predicted, test_Y['category'])
    f1 = f1_score(dev_predicted, test_Y['category'], average='weighted')
    precision = precision_score(dev_predicted, test_Y['category'], average='weighted')
    recall = recall_score(dev_predicted, test_Y['category'], average='weighted')

    print(f'epoch: {e}, lr: {scheduler.get_last_lr()[0]:.8f}, grad_norm: {np.mean(grad_norms)}, training_loss: {np.mean(training_losses):.4f}, validation_loss: {np.mean(validation_losses)}, accuracy: {accuracy}, f1: {f1}, precision: {precision}, recall: {recall}')

    # Call scheduler.step() with the f1 score
    scheduler.step(f1)

    if f1 >= best_dev_acc:
        best_dev_acc = f1
        current_patient = 0
        model.save_pretrained('debertav3-finetune-banking-transaction-classification')
    else:
        current_patient += 1

    if current_patient >= patient:
        break

    if e == epoch - 1:
        print(f'Final dev f1-score: {best_dev_acc}')

100%|██████████| 3114/3114 [13:57<00:00,  3.72it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch: 0, lr: 0.00001000, grad_norm: 75.29785482073412, training_loss: 2.2216, validation_loss: 2.0496841600219766, accuracy: 0.4196442909908463, f1: 0.49923485519130106, precision: 0.9076706288344336, recall: 0.4196442909908463


100%|██████████| 3114/3114 [13:56<00:00,  3.72it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch: 1, lr: 0.00001000, grad_norm: 80.91857140356711, training_loss: 1.9896, validation_loss: 1.9489452177814097, accuracy: 0.42550586157057974, f1: 0.5008338185973946, precision: 0.9098833324133618, recall: 0.42550586157057974


100%|██████████| 3114/3114 [13:55<00:00,  3.73it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

epoch: 12, lr: 0.00001000, grad_norm: 217.90096990619628, training_loss: 0.5098, validation_loss: 0.5460153952986958, accuracy: 0.8788943311385901, f1: 0.9050514098191573, precision: 0.9466550932146042, recall: 0.8788943311385901


100%|██████████| 3114/3114 [13:55<00:00,  3.73it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch: 13, lr: 0.00001000, grad_norm: 229.62083500305965, training_loss: 0.4696, validation_loss: 0.5186495848659985, accuracy: 0.8835113216637225, f1: 0.9077468495208842, precision: 0.9477574346613298, recall: 0.8835113216637225


100%|██████████| 3114/3114 [13:54<00:00,  3.73it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch: 14, lr: 0.00001000, grad_norm: 264.8911172711244, training_loss: 0.4406, validation_loss: 0.49374220320364326, accuracy: 0.8831901397141481, f1: 0.9076078994662509, precision: 0.9476849973108531, recall: 0.8831901397141481
Final dev accuracy: 0.9077468495208842


In [ ]:
label2id = {category: index for index, category in enumerate(sorted_categories)}
id2label = {index: category for index, category in enumerate(sorted_categories)}

In [ ]:
model.config.label2id = label2id
model.config.id2label = id2label

In [ ]:
print(
    classification_report(
        dev_predicted, test_Y['category'], target_names = sorted(category),
        digits = 5
    )
)

                            precision    recall  f1-score   support

                       ATM    0.99824   0.96095   0.97924      1178
    Arts and Entertainment    0.35897   0.96552   0.52336        29
                  Bank Fee    0.00000   0.00000   0.00000         0
                 Bank Fees    0.98829   0.98984   0.98906      1279
             Check Deposit    0.90476   0.90476   0.90476        42
  Clothing and Accessories    0.71593   0.86337   0.78276       505
        Convenience Stores    0.97664   0.56963   0.71957      5946
         Department Stores    0.02611   1.00000   0.05089        10
     Digital Entertainment    0.97457   0.90841   0.94032       928
Food and Beverage Services    0.01471   0.50000   0.02857         2
              Gas Stations    0.00304   0.38889   0.00603        18
  Gyms and Fitness Centers    0.07143   1.00000   0.13333         1
                Healthcare    0.10811   0.44444   0.17391         9
                 Insurance    0.04281   0.77778

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 4.0 Save and upload to HuggingFace

In [ ]:
create_repo("wanadzhar913/debertav3-finetuned-banking-transaction-classification", repo_type="model")

RepoUrl('https://huggingface.co/wanadzhar913/debertav3-finetuned-banking-transaction-classification', endpoint='https://huggingface.co', repo_type='model', repo_id='wanadzhar913/debertav3-finetuned-banking-transaction-classification')

In [ ]:
model.push_to_hub('wanadzhar913/debertav3-finetuned-banking-transaction-classification', safe_serialization = True)

model.safetensors:   0%|          | 0.00/740M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wanadzhar913/debertav3-finetuned-banking-transaction-classification/commit/a8b4144c9a83250b6fe7e4ebe4df22e48b5e83d1', commit_message='Upload CustomSequenceClassification', commit_description='', oid='a8b4144c9a83250b6fe7e4ebe4df22e48b5e83d1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/wanadzhar913/debertav3-finetuned-banking-transaction-classification', endpoint='https://huggingface.co', repo_type='model', repo_id='wanadzhar913/debertav3-finetuned-banking-transaction-classification'), pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub('wanadzhar913/debertav3-finetuned-banking-transaction-classification', safe_serialization = True)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wanadzhar913/debertav3-finetuned-banking-transaction-classification/commit/3d03da6dbc65f4a80bb57e080afddd68b904995c', commit_message='Upload tokenizer', commit_description='', oid='3d03da6dbc65f4a80bb57e080afddd68b904995c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/wanadzhar913/debertav3-finetuned-banking-transaction-classification', endpoint='https://huggingface.co', repo_type='model', repo_id='wanadzhar913/debertav3-finetuned-banking-transaction-classification'), pr_revision=None, pr_num=None)

### 5.0 How to use the model

In [12]:
from typing import Optional, Union, Tuple

import pandas as pd
from datasets import load_dataset, Dataset

import torch
from torch import nn
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import AutoTokenizer, DebertaV2Model, \
                         DebertaV2ForSequenceClassification

In [5]:
!mkdir data/
!cd data/ && wget https://raw.githubusercontent.com/wanadzhar913/bank-transaction-classification/refs/heads/master/data/test.csv -q

In [6]:
# load sample dataset
test = pd.read_csv('data/test.csv')
df_test = Dataset.from_pandas(test)

In [7]:
cols_to_remove = ['client_id', 'bank_id', 'account_id', 'txn_id', 'txn_date',  'description_stem', 'description_clean_len', 'category']
cols_to_preprocess = sorted(list(set(df_test.column_names) - set(cols_to_remove)))

In [8]:
def transform_row(row):
    """This is a function to extract the text data and convert our non-text features into a list."""

    # Separate description
    description = row["description"]

    # Collect remaining features and convert to a FloatTensor
    extra_data = [row[key] for key in cols_to_preprocess if key != "description"]

    return {
        "description": description,
        "extra_data": extra_data, # for all the extra columns
    }

In [16]:
test_X = df_test.remove_columns(cols_to_remove) \
                .map(transform_row, num_proc=2) \
                .select_columns(['description', 'extra_data'])

Map (num_proc=2):   0%|          | 0/49816 [00:00<?, ? examples/s]

In [10]:
class ClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config, num_extra_dims):
        super().__init__()
        total_dims = config.hidden_size+num_extra_dims
        self.dense = nn.Linear(total_dims, total_dims)
        classifier_dropout = config.hidden_dropout_prob
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(total_dims, config.num_labels)

    def forward(self, features, **kwargs):
        x = self.dropout(features)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

In [13]:
class CustomSequenceClassification(DebertaV2ForSequenceClassification):

    def __init__(self, config, num_extra_dims):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        # might need to rename this depending on the model
        self.deberta =  DebertaV2Model(config)
        self.classifier = ClassificationHead(config, num_extra_dims)

        # Initialize weights and apply final processing
        self.post_init()


    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        extra_data: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.deberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # sequence_output will be (batch_size, seq_length, hidden_size)
        sequence_output = outputs[0]

        # additional data should be (batch_size, num_extra_dims)
        cls_embedding = sequence_output[:, 0, :]

        output = torch.cat((cls_embedding, extra_data), dim=-1)

        logits = self.classifier(output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = nn.MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = nn.BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [49]:
model = CustomSequenceClassification.from_pretrained(
    "wanadzhar913/debertav3-finetuned-banking-transaction-classification",
    num_labels=33,
    num_extra_dims=44,
)

_ = model.eval().cuda()

In [15]:
tokenizer = AutoTokenizer.from_pretrained('wanadzhar913/debertav3-finetuned-banking-transaction-classification')

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [67]:
padded = tokenizer(test_X['description'][0], padding = 'longest', return_tensors = 'pt')
for k in padded.keys():
    padded[k] = padded[k].cuda()

with torch.no_grad():
    pred = model(
        **padded,
        extra_data=torch.FloatTensor([test_X['extra_data'][0]]).cuda(),
        return_dict = False
    )
    # output is 'Restaurants' (refer to `config.json`)
    print(pred[0].argmax(axis = 1).detach().cpu().numpy()[0])

19
